In [1]:
# Import necessary libraries
from openprompt.plms import LMTokenizerWrapper
from datasets import load_from_disk
from openprompt.pipeline_base import PromptDataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from openprompt.prompts import PrefixTuningTemplate
from openprompt import PromptForClassification
from openprompt.data_utils import FewShotSampler
from random import shuffle
import torch
from transformers import AdamW
from transformers.optimization import get_linear_schedule_with_warmup
from openprompt.data_utils import InputExample
import json
dataset_path = "/lustre/work/client/users/minhos/cache/datasets/p3_sciq_multiple_choice"
raw_dataset = load_from_disk(dataset_path)

label_map = {"positive": 0,"negative": 1}

dataset = {}
for split in ['train', 'validation']:
    dataset[split] = []
    if split == 'train':
        raw_dataset[split] = raw_dataset[split].shuffle(seed=42).select(range(1000))
    else:
        raw_dataset[split] = raw_dataset[split].select(range(1000))
    
    for idx, data in enumerate(raw_dataset[split]):
        label_text = data["targets_pretokenized"].strip().lower()
        label_numeric = label_map.get(label_text, -1)
        input_example = InputExample(text_a=data['inputs_pretokenized'], guid=idx, label=label_numeric)
        dataset[split].append(input_example)
print(dataset['train'][0])
print(type(dataset['train'][0]))

sampler = FewShotSampler(num_examples_per_label=30)
fewshot_data = sampler(dataset['train'], seed=42)

# Load the T5 model
from openprompt.plms import load_plm
gpt_path = "/lustre/work/client/users/minhos/models_for_supercomputer/gpt2"
model = GPT2LMHeadModel.from_pretrained(gpt_path)
tokenizer = GPT2Tokenizer.from_pretrained(gpt_path)
tokenizer.pad_token = tokenizer.eos_token 



# Logging setup
log_file = "prefix_tuning_results_gpt2_6.json"
results = []

# Hyperparameter search ranges
learning_rates = [0.005] # 0.0005, 0.001, 0.005
num_soft_tokens = [50, 100] # 10, 50, 100
warmup_steps = [10, 20, 25] #10, 20, 25


for lr in learning_rates:
    for tokens in num_soft_tokens:
        for warmup in warmup_steps:
            print(f"Testing: LR={lr}, Soft Tokens={tokens}, Warm-Up Steps={warmup}")
            
            model = GPT2LMHeadModel.from_pretrained(gpt_path)
            tokenizer = GPT2Tokenizer.from_pretrained(gpt_path)
            tokenizer.pad_token = tokenizer.eos_token 
            template = PrefixTuningTemplate(
                model=model,
                tokenizer=tokenizer,
                text='{"placeholder":"text_a"} {"mask"}',
                num_token=10,  # Number of virtual tokens
            )
            
            from openprompt.prompts import ManualVerbalizer
            verbalizer = ManualVerbalizer(
                tokenizer=tokenizer, 
                num_classes=2,  # Example: binary classification
                label_words=[["positive", "good", "excellent", "wonderful"], ["negative", "bad", "horrible", "terrible"]],
                classes= [0, 1]
            )
            
            wrapped_example = template.wrap_one_example(dataset['train'][0])


            prompt_model = PromptForClassification(plm=model,template=template, verbalizer=verbalizer, freeze_plm=True)
            
            
            train_dataloader = PromptDataLoader(dataset=fewshot_data, template=template, tokenizer=tokenizer,
                tokenizer_wrapper_class=LMTokenizerWrapper, max_seq_length=480, decoder_max_length=3,
                batch_size=5,shuffle=True, teacher_forcing=False, predict_eos_token=False,
                truncate_method="tail")
            
            validation_dataloader = PromptDataLoader(dataset=dataset["validation"], template=template, tokenizer=tokenizer,
                tokenizer_wrapper_class=LMTokenizerWrapper, max_seq_length=480, decoder_max_length=3,
                batch_size=5,shuffle=False, teacher_forcing=False, predict_eos_token=False,
                truncate_method="tail")
            
            # Define loss function
            loss_func = torch.nn.CrossEntropyLoss()
            
            # Define optimizer for the prefix tuning parameters
            optimizer_grouped_parameters = [{'params': [p for name, p in template.named_parameters() if 'raw_embedding' not in name]}]
            optimizer = AdamW(optimizer_grouped_parameters, lr=lr)
            
            # Define a learning rate scheduler
            scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=10, num_training_steps=1000)


            from tqdm import tqdm
            
            # Ensure model is in training mode
            prompt_model.train()
            
            # Training parameters
            num_epochs = 10
            gradient_accumulation_steps = 1
            
            for epoch in range(num_epochs):
                print(f"Epoch {epoch + 1}/{num_epochs}")
                total_loss = 0
                pbar = tqdm(train_dataloader, desc="Training")
                
                for step, inputs in enumerate(pbar):
                    logits = prompt_model(inputs)
                    labels = inputs['label']  # Ground-truth labels
                    
                    # Compute loss
                    loss = loss_func(logits, labels)
                    loss.backward()  # Backpropagation
                    
                    # Optimizer step
                    if (step + 1) % gradient_accumulation_steps == 0:
                        optimizer.step()
                        scheduler.step()
                        optimizer.zero_grad()
                    
                    total_loss += loss.item()
                    pbar.set_postfix({"loss": total_loss / (step + 1)})
            
            
            def evaluate(prompt_model, dataloader):
                prompt_model.eval()  # Set the model to evaluation mode
                total, correct = 0, 0
                
                with torch.no_grad():
                    for inputs in dataloader:
                        logits = prompt_model(inputs)
                        preds = torch.argmax(logits, dim=-1)
                        labels = inputs['label']
                        
                        total += len(labels)
                        correct += (preds == labels).sum().item()
                
                accuracy = correct / total
                return accuracy
            
            
            # Validation after each epoch
            val_accuracy = evaluate(prompt_model, validation_dataloader)
            print(f"Validation Accuracy after Epoch {epoch + 1}: {val_accuracy:.4f}")
            # Log results
            result = {
                "learning_rate": lr,
                "num_soft_tokens": tokens,
                "warmup_steps": warmup,
                "final_loss": total_loss / (10 * len(train_dataloader)),
                "accuracy": val_accuracy
            }
            results.append(result)

            # Save intermediate results
            with open(log_file, "w") as f:
                json.dump(results, f, indent=4)

                    
print("Tuning complete. Results saved to", log_file)


/lustre/work/client/users/minhos/.conda/envs/prompt_learning_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/lustre/work/client/users/minhos/.conda/envs/prompt_learning_env/lib/python3.10/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


{
  "guid": 0,
  "label": 1,
  "meta": {},
  "text_a": ". . . plays like somebody spliced random moments of a chris rock routine into what is otherwise a cliche-riddled but self-serious spy thriller . What is the sentiment expressed by the reviewer for the movie? ",
  "text_b": "",
  "tgt_text": null
}

<class 'openprompt.data_utils.utils.InputExample'>


/lustre/work/client/users/minhos/.conda/envs/prompt_learning_env/lib/python3.10/site-packages/transformers/modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

Testing: LR=0.005, Soft Tokens=50, Warm-Up Steps=10


tokenizing: 60it [00:00, 1340.52it/s]
tokenizing: 1000it [00:00, 1841.05it/s]
/lustre/work/client/users/minhos/.conda/envs/prompt_learning_env/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

Epoch 9/10


Training: 100%|██████████| 12/12 [00:44<00:00,  3.67s/it, loss=0.712]


Epoch 10/10


Training: 100%|██████████| 12/12 [00:42<00:00,  3.55s/it, loss=0.731]


Validation Accuracy after Epoch 10: 0.4670
Tuning complete. Results saved to prefix_tuning_results_gpt2_6.json


In [ ]:
# This time the verbalizer label: label_words=[["positive", "good", "excellent", "wonderful"], ["negative", "bad", "horrible", "awful"]]
# loss in the beginning: 0.829
# loss after training 10 epochs: 0.632
# Number of soft tokens: 10
# warmup step: 10
# Number of samples per label: 30 
# learning rate=1e-4
# accuracy:0.5180


# ----------
# loss in the beginning: 0.955
# loss after training 10 epochs: 0.698
# warmup step: 10
# Number of soft tokens: 10
# Number of samples per label: 30 
# learning rate=0.005
# accuracy:0.5330
